In [1]:
import csv
import json
import requests
import numpy as np
import pandas as pd
import os

## Load data and map champion ID to Name and alphabetical index

In [2]:
filename = '../data/champion.json'
with open(filename, 'r') as f:
    champ_data = json.load(f)

# contains index of championId and name
id_champ_map = {}
champion_list = []
for i, c in enumerate(champ_data['data'].keys()):
    id_champ_map[int(champ_data['data'][c]['key'])] = {'champion': champ_data['data'][c]['id'], 'index': i}
    champion_list.append(champ_data['data'][c]['id'])

## Save data in matrix format for analysis and df

In [3]:
filename = '../data/complete_mastery_info.json'

with open(filename, 'r') as f:
    data = json.load(f)

names = []
r = np.zeros((len(data.keys()), len(champion_list)))

for i, key in enumerate(data.keys()):
    names.append([key])
    for j, entry in enumerate(data[key]):
        if j == 0:
            maximum = entry['championPoints']
        r[i][id_champ_map[entry['championId']]['index']] = entry['championPoints']/maximum

        fields = ['player_id']
print(r)
with open('../data/names.csv', 'w') as f:
    write = csv.writer(f)
     
    write.writerow(fields)
    write.writerows(names)

np.savetxt('../data/mastery.csv', r, delimiter=',', header=','.join(champion_list))

[[7.61825457e-02 5.18862906e-02 3.54358939e-03 ... 4.90366155e-04
  6.36134434e-02 1.36423565e-02]
 [1.00000000e+00 9.05759162e-02 3.69982548e-02 ... 0.00000000e+00
  0.00000000e+00 9.16230366e-02]
 [2.53620687e-02 3.83951623e-02 3.18648093e-02 ... 0.00000000e+00
  0.00000000e+00 1.80861786e-03]
 ...
 [0.00000000e+00 0.00000000e+00 6.89265682e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.14366935e-03 1.85969249e-02 3.31635932e-03 ... 2.61314085e-02
  2.96435458e-03 5.48415484e-02]
 [0.00000000e+00 2.65633647e-04 3.78527947e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


## Save data in surprise format to load into surprise
The dataset has 5326 users from the ranked pool. For each user, the dataset contains the user's 25 most played champions and each champion's corresponding mastery score for the user. Champion mastery score is highly correlated with playtime.

I loaded in champion mastery data and set each user's highest played champion's mastery score to 1. I then divide every other champion's mastery point score by the highest played champion's mastery point score. We assume that any champion after the 25th has a rating of 0. This is a crude approximation since the mean of the 25th highest mastery score was 0.09. The rating scale is therefore 0 to 1.

In [4]:
filename = '../data/complete_mastery_info.json'

with open(filename, 'r') as f:
    data = json.load(f)

ratings = [] # list of items
total = 0
count = 0
for i, key in enumerate(data.keys()):
    for j, entry in enumerate(data[key]):
        if j == 0:
            # sorted in descending order
            maximum = entry['championPoints']
        if j == 50:
            total += entry['championPoints']/maximum
            count += 1

print(f'Mean of 25th highest champion score: {total/count}')

Mean of 25th highest champion score: 0.04109415751001418


In [5]:
filename = '../data/complete_mastery_info.json'

with open(filename, 'r') as f:
    data = json.load(f)

ratings = [] # list of items
r = np.zeros((len(data.keys()), len(champion_list)))

for i, key in enumerate(data.keys()):
    used = []
    for j, entry in enumerate(data[key]):
        if j == 0:
            # sorted in descending order
            maximum = entry['championPoints']
        score = entry['championPoints']/maximum
        item = id_champ_map[entry['championId']]['champion']
        ratings.append([key, item, score])
        used.append(item)
    # set unknown champion IDs to zero
    for champ in champion_list:
        if champ not in used:
            ratings.append([key, champ, 0])
    else:
        continue

print(f'users: {i}')
print(f'ratings: {len(ratings)}')
with open('../data/surprise_data_full.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(ratings)

users: 2896
ratings: 478005


In [6]:
# save first 750 users for quicker analysis

filename = '../data/complete_mastery_info.json'

with open(filename, 'r') as f:
    data = json.load(f)

ratings = [] # list of items
r = np.zeros((len(data.keys()), len(champion_list)))

for i, key in enumerate(data.keys()):
    if i >= 750:
        break
    used = []
    for j, entry in enumerate(data[key]):
        if j == 0:
            # sorted in descending order
            maximum = entry['championPoints']
        score = entry['championPoints']/maximum
        item = id_champ_map[entry['championId']]['champion']
        ratings.append([key, item, score])
        used.append(item)
    # set unknown champion IDs to zero
    for champ in champion_list:
        if champ not in used:
            ratings.append([key, champ, 0])
    else:
        continue

print(f'users: {i}')
print(f'ratings: {len(ratings)}')
with open('../data/surprise_data_750.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(ratings)

users: 750
ratings: 123750
